In [23]:
# Install necessary packages
!pip install -q MRdataset
!pip install -q mrQA

In [24]:
# Import required libraries
import IPython
from MRdataset import import_dataset
from mrQA import check_compliance

In [25]:
# Define data paths
data_root = '/home/jovyan/shared/ds003688'  # Change path here
output_dir = '/home/jovyan/group_project/mrqa_reports/'  # Change path here

In [26]:
# Import the dataset
dataset = import_dataset(data_source=data_root,
                         ds_format='bids',
                         name='ds003688')

In [27]:
# Print dataset information
print(dataset)

FastBIDSDataset ds003688 with 2 Modality


In [28]:
# Print modalities and subjects
for modality in dataset.modalities:
    print(modality)

Modality anat_T1w with 63 Subject
Modality func_task-film_bold with 30 Subject


In [29]:
# Check compliance and generate a report
check_compliance(dataset=dataset, output_dir=output_dir)

In ds003688 dataset, modalities "anat_T1w" are non-compliant. See /home/jovyan/group_project/mrqa_reports/ds003688_DATE_08_15_2023_05_14_58.html for report


PosixPath('/home/jovyan/group_project/mrqa_reports/ds003688_DATE_08_15_2023_05_14_58.html')